# Results on basic usecases analysis


# 0. Imports

In [4]:
import pandas as pd
import plotly.express as px

## 1. Basic Autotuning

### 1.1 Generating new data

In [2]:
import subprocess

script = "../scripts/test_basic_AT.sh"

# Exécution du script shell
try:
    subprocess.run(["sh", script], check=True)
except subprocess.CalledProcessError as e:
    print("Error :", e)

Une erreur s'est produite lors de l'exécution du script : Command '['sh', '../script/test_basic_AT.sh']' returned non-zero exit status 2.


sh: 0: cannot open ../script/test_basic_AT.sh: No such file


### 1.2 Analysis 

In [8]:
df = pd.read_csv("../data/trace_basic_autotuning.csv",sep = ";")

,i,param_value,time_recorded
0,0,100,149816140
1,1,1000000000,2464214839
2,2,10,121222872
3,3,10,119671970
4,4,10,9494
...,...,...,...
1505,5,10,1935
1506,6,10,1715
1507,7,10,1764
1508,8,10,1793


### 1.2.1 Most chosen i within {10,100000,100000000}

In [10]:
df_i4 = df[df['i'] == 4]

count_per_param_value = df_i4['param_value'].value_counts().reset_index()
count_per_param_value.columns = ['param_value', 'count']
count_per_param_value["param_value"] = count_per_param_value["param_value"].astype("str")
fig = px.bar(count_per_param_value, x='param_value', y='count', 
             title='Nombre de lignes avec param_value identiques pour i = 4',
             labels={'param_value': 'param_value distinct', 'count': 'Nombre de lignes'})
fig.show()

In [15]:
average_recorded_time = df.groupby('i')['time_recorded'].mean()
average_recorded_time

i
0    1.662733e+08
1    2.503687e+09
2    1.324014e+08
3    1.271833e+08
4    1.189170e+04
5    2.288272e+03
6    2.083285e+03
7    2.170894e+03
8    1.999397e+03
9    1.973351e+03
Name: time_recorded, dtype: float64

## 1.3 Conclusion

This very simple example shows how well our jited autotuning method works.   
We always select the loop with the fewest iterations.   
Nevertheless, we can see that the compilation time is very, very long compared with the execution time once compiled.

# 2. Blocksize matmul

## 2.1 Generating new data

In [ ]:
script = "../scripts/blockMatmul.sh"

try:
    subprocess.run(["sh", script], check=True)
except subprocess.CalledProcessError as e:
    print("Error :", e)

In [28]:
df = pd.read_csv("../data/trace_blocksizeMatmul.csv",sep = ";")
filtered_df = df[df['i'] == 9]
df["N"] = df["N"].astype('str')
df["taille_block"] = df["taille_block"].astype('str')

heatmap_data = filtered_df.pivot_table(index='N', columns='taille_block', values='i', aggfunc='count', fill_value=0)


fig = px.imshow(heatmap_data,
                x=heatmap_data.columns,
                y=heatmap_data.index,
                labels=dict(x='Taille Block', y='N', color='Nombre de taille_block'),
                color_continuous_scale='viridis')


fig.update_layout(title='Heatmap pour i = 9',
                  xaxis_title='Taille Block',
                  yaxis_title='N',
                  width = 1000)

fig.show()


In [27]:
df

,i,taille_block,time_recorded,N
0,0,2,223181613,8
1,1,4,196255162,8
2,2,8,198061204,8
3,3,16,199688424,8
4,4,32,205289768,8
...,...,...,...,...
1006,6,128,7923924832736,8096
1007,7,256,7855524085927,8096
1008,8,512,8035837668465,8096
1009,9,256,8001040636543,8096


In [22]:
heatmap_data

taille_block,4,8,16,32,64,128,256,512
N,,,,,,,,
8,0,1,1,2,1,1,0,4
16,1,1,2,2,1,1,2,0
32,0,2,0,0,2,3,1,2
64,0,0,2,3,1,0,1,3
128,0,1,1,2,1,1,4,0
256,0,0,1,2,0,3,1,3
512,0,0,1,0,2,0,4,3
1024,0,0,0,1,0,3,5,1
2048,0,0,0,1,0,4,4,1


# 3. Selecting Matmul analysis

In [31]:
df = pd.read_csv("../data/trace_selecting_matmul.csv",sep = ";")
df

,nb_runs,i,matmul_function,time_recorded
0,10,0,0,479212816
1,10,1,1,233864642
2,10,2,2,215701079
3,10,3,2,226934659
4,10,4,2,27864
...,...,...,...,...
1135,10,5,1,4647
1136,10,6,1,4191
1137,10,7,1,4217
1138,10,8,1,4254


In [34]:
df_chosen_one = df[df.i == 4]
px.bar(df_chosen_one["matmul_function"].value_counts())